# Verify

In [3]:
print("Hello Slicer")

Hello Slicer


# Setup

In [4]:
import numpy as np
import JupyterNotebooksLib as slicernb
import logging
import time
import sys
import pickle
import json
from collections import deque
from queue import PriorityQueue

from importlib import reload

reload(logging)

<module 'logging' from '/Applications/Slicer4.app/Contents/lib/Python/lib/python3.6/logging/__init__.py'>

In [5]:
# create logger
logger = logging.getLogger('scratch_logger')
logger.setLevel(logging.DEBUG)

logstream_ch = logging.StreamHandler()
logstream_ch.setLevel(logging.INFO)

logfile_ch = logging.FileHandler('bfs.log')
logfile_ch.setLevel(logging.DEBUG)

logger.addHandler(logstream_ch)
logger.addHandler(logfile_ch)

In [6]:
loadScene("data/slicer_gs_clean_update1/2023-10-26-Scene.mrml")

In [7]:
# # create debug console
# console = qt.QTabWidget()
# console_tab1 = qt.QWidget()
# console_tab1_layout = qt.QHBoxLayout(console_tab1)
# btnA = qt.QPushButton('Calculate')
# btnB = qt.QPushButton('B')
# console_tab1_layout.addWidget(btnA)
# console_tab1_layout.addWidget(btnB)

# console.addTab(console_tab1, 'tab1')

# console.show()

# btnA.disconnect('clicked()', calculate_backbone)
# btnA.connect('clicked()', calculate_backbone)

# # btnA.disconnect('clicked()', btnA_action)
# # btnA.connect('clicked()', btnA_action)

# # btnB.disconnect('clicked()', btnB_action)
# # btnB.connect('clicked()', btnB_action)

pass

In [8]:
# slicernb.ViewDisplay()
# slicernb.ViewLightboxDisplay(columns=5, rows=3, rangeShrink=[15,20])
pass

In [ ]:
# # Clear scene
# slicer.mrmlScene.Clear(False)

# Load from local file
# volume = slicer.util.loadVolume('data/MRBrainTumor1.nrrd')


# # Lightbox view
# slicernb.ViewLightboxDisplay("Red", rangeShrink=[45,50])
pass

## Global

In [7]:
def vtk4x4matrix_to_numpy(matrix):
    """
    Copies the elements of a vtkMatrix4x4 into a numpy array.

    :param matrix: The matrix to be copied into an array.
    :type matrix: vtk.vtkMatrix4x4
    :rtype: numpy.ndarray
    """
    m = np.ones((4, 4))
    for i in range(4):
        for j in range(4):
            m[i, j] = matrix.GetElement(i, j)
    return m

VOLUME_NODE_NAME = "14: Body Soft Tissue"
OUT_A_NODE_NAME = "out_a"
OUT_B_NODE_NAME = "out_b"
OUT_C_NODE_NAME = "out_c"
MYNULL = -2_147_483_648

volume_node = getNode(VOLUME_NODE_NAME)
volume_display = volume_node.GetDisplayNode()
volume_display.InterpolateOff()

outa_node = getNode(OUT_A_NODE_NAME)
outa_node.GetDisplayNode().InterpolateOff()

outb_node = getNode(OUT_B_NODE_NAME)
outb_node.GetDisplayNode().InterpolateOff()

outc_node = getNode(OUT_C_NODE_NAME)
outc_node.GetDisplayNode().InterpolateOff()

lad_node = getFirstNodeByName("LAD")
lcx_node = getFirstNodeByName("LCX")
rca_node = getFirstNodeByName("RCA")

v_arr = slicer.util.array(VOLUME_NODE_NAME)
print(v_arr.shape)
print(v_arr.min())
print(v_arr.max())
print(v_arr.dtype)

x = vtk.vtkMatrix4x4()
volume_node.GetIJKToRASMatrix(x)
ijk2ras_mat = vtk4x4matrix_to_numpy(x)

volume_node.GetRASToIJKMatrix(x)
ras2ijk_mat = vtk4x4matrix_to_numpy(x)

# ras2ijk_mat @ np.array([326,192,160,1]).reshape(-1,1)

(275, 512, 512)


-3024
1960
int32


## Util

In [8]:
def get_fiducial_as_kji(fiducial_node, i):
    ras_homo = np.ones([4,1])
    fiducial_node.GetNthFiducialPosition(i, ras_homo[:3,0])
    return tuple((np.round( ras2ijk_mat @ ras_homo ).astype(np.uint16).T)[(0,0,0),(2,1,0)].tolist())

def get_fiducial_data(fiducial_node, i):
    ras_homo = np.ones([4,1])
    fiducial_node.GetNthFiducialPosition(i, ras_homo[:3,0])
    posk,posj,posi = (np.round( ras2ijk_mat @ ras_homo ).astype(np.uint16).T)[(0,0,0),(2,1,0)].tolist()
    r,a,s = ras_homo[:3,0]
    lb = fiducial_node.GetNthControlPointLabel(i)
    return (lb, r, a, s, posi, posj, posk)

def ijk2kji(ijk):
    return (ijk[2], ijk[1], ijk[0])

def multi_add_fiducial(fid_node, nodes_ras_Nx3):
    for i in range(len(nodes_ras_Nx3)):
        fid_node.AddFiducialFromArray(nodes_ras_Nx3[i,:])
        
def coord2z(i,j,k):
    return i * (262144) + j * (512) + k
def z2coord(z):
    # a[i//(6*7),(i%(6*7)//7),i%(7)]
    return z//(512*512), z%(262144)//512, z%512

In [9]:
# create segmentation node to store result
# seg_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")

# seg_node.SetName("segmentation_result")
# labelmap_node = getNode('labelmap_template')
pass

In [10]:
# import MRMLCorePython
# labelmap_node = MRMLCorePython.vtkMRMLLabelMapVolumeNode()
pass

In [11]:
# def create_result(in_arr):
#     # seg_arr = np.zeros(v_arr.shape)
#     # seg_arr[100:,:,:] = 1
#     seg_arr = in_arr.copy().astype('int16')
#     updateVolumeFromArray(labelmap_node, seg_arr)
#     slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap_node, seg_node)
pass

In [12]:
# d_arr = v_arr.copy()
# r_arr = v_arr.copy()
# del r_arr
pass

# Data Analysis Export

In [18]:
import pandas as pd

In [20]:
lad_node

,label,position.R,position.A,position.S,selected,visible,description
0,LAD-1,-21.565567,45.475788,-386.50,True,True,
1,LAD-2,-25.112342,46.630552,-386.50,True,True,
2,LAD-3,-31.710992,50.136085,-389.00,True,True,
3,LAD-4,-34.318446,51.385463,-390.25,True,True,
4,LAD-5,-36.260090,53.118542,-391.50,True,True,
5,LAD-6,-37.693939,54.910854,-392.75,True,True,
6,LAD-7,-40.442150,57.778551,-395.25,True,True,
7,LAD-8,-43.548822,60.526762,-397.75,True,True,
8,LAD-9,-46.476264,63.215228,-399.00,True,True,


In [23]:
pda_col_list = ['label', 'artery', 'R', 'A', 'S', 'k', 'j', 'i', 'hu']
pda_data_map = { e : [] for e in pda_col_list }
for artery_fid_node, artery_name in zip([lad_node, lcx_node, rca_node], ['LAD', 'LCX', 'RCA']):
    n = artery_fid_node.GetNumberOfControlPoints()
    for i in range(n):
        label, pos_r, pos_a, pos_s, pos_i, pos_j, pos_k = get_fiducial_data(artery_fid_node, i)
        # label = artery_fid_node.GetNthControlPointLabel(i)
        # print(label)
        hu = v_arr[pos_k,pos_j,pos_i]
        pda_data_map['label'].append(label)
        pda_data_map['artery'].append(artery_name)
        pda_data_map['R'].append(pos_r)
        pda_data_map['A'].append(pos_a)
        pda_data_map['S'].append(pos_s)
        pda_data_map['i'].append(pos_i)
        pda_data_map['j'].append(pos_j)
        pda_data_map['k'].append(pos_k)
        pda_data_map['hu'].append(hu)


df = pd.DataFrame(pda_data_map)
df.to_csv('data/pda/fiducial_markers_data.csv')
np.save('data/pda/volume_array.npy', v_arr)


In [ ]:
d_arr = v_arr.copy()
d_arr[d_arr > -200] = MYNULL
d_arr[d_arr < -1050] = MYNULL
d_arr[d_arr != MYNULL] = 500

updateVolumeFromArray(canvas_node, d_arr)

In [ ]:
# start_point = [0,0,0] 
# getNode("LAD_2").GetNthFiducialPosition(0, start_point)
# # start_point = (start_point * np.array([-1, 1, -1])).astype("int32")
# start_point = (start_point * np.array([1, 1, 1])).astype("int32")
# i,j,k = start_point

# i,j,k = [162, 198, 313]

# d_arr = v_arr.copy()
# d_arr[:] = MYNULL
# d_arr[i-50:i+50,j-50:j+50,k-50:k+50] = 500
# updateVolumeFromArray(canvas_node, d_arr)


In [ ]:
# numpy data format
# i - z 0 is downmost
# j - y 0 is topmost
# k - x 0 is leftmost
pass

In [ ]:
pos_a = np.arange(5*6*7).reshape(5,6,7)
for pos_i in range(5*6*7):
    print(pos_a[pos_i//(6*7),(pos_i%(6*7)//7),pos_i%(7)])


In [ ]:
# visualize 
d_arr = v_arr.copy()
d_arr[:] = MYNULL
d_arr[85:186,115:256,240:401] = 2000

updateVolumeFromArray(canvas_node, d_arr)

In [ ]:
# gradient filter
d_arr = v_arr.copy()
grads = np.gradient(d_arr)

d_arr = np.sqrt(grads[0]**2 + grads[1]**2 + grads[2]**2)
# d_arr = d_arr/100
updateVolumeFromArray(canvas_node, d_arr)

print(np.min(d_arr))
print(np.max(d_arr))

# Flood Fill

In [13]:
EXP_NAME = 'default EXP NAME'

# inclusive bound
bound = np.array([
    [85, 185],
    [115, 255],
    [240, 401]
])

def is_valid_index(i,j,k):
    # bounding
    return i >= 85 and j >= 115 and k >= 240 and i < 186 and j < 256 and k < 401
    return i >= 0 and j >= 0 and k >= 0 and i < 275 and j < 512 and k < 512 


# bfs

# LAD
seed = (162, 198, 313)
end_node = (152, 175, 345)

# LCX
seed = (160, 204, 306)
end_node = (151, 212, 303)

# RCA
seed = (93, 156, 282)
end_node = (122, 138, 274)

t_start = time.time()

queue = [seed]
visited = {seed}
mask_index = {seed}
# grid = np.array([])
visit_order = {
    seed : 0
}


# offsets = []
# for i in [-1,0,1]:
#     for j in [-1,0,1]:
#         for k in [-1,0,1]:
#             if i != 0 or j != 0 or k != 0:
#                 offsets.append(np.array([i,j,k]))
kernel_mask = np.zeros((5,5,5))
kernel_mask[1:4,1:4,1:4] = 1
kernel_mask[2,2,2] = 0

kernel_idxs = np.vstack(np.where(kernel_mask == 1)) - np.array([[2],[2],[2]])
# offsets = []

r_arr = v_arr.copy()
it = 0

is_reach = False

while len(queue) and it < 2e6:
    it += 1
    v = queue.pop(0)
    
    vi, vj, vk = v
    vb = r_arr[vi, vj, vk]
    
    pos = np.array(v)
    
    # debug
    next_visit_order = visit_order[v] + 1
    
    for pos_i in range(kernel_idxs.shape[1]):
        w = tuple((kernel_idxs[:,pos_i] + pos).tolist())
        wi, wj, wk = w
        
        # logger.debug(w)
        if not is_valid_index(wi, wj, wk):
            # print(wi, wj, wk)
            continue
        if w in visited:
            continue
           
        
        wb = r_arr[wi, wj, wk]
        diff = abs(vb - wb)
        if (wb < -300 and wb > -1200 and (diff < 250 or wb < -400)) \
            or wb > 150: # calcium
            visited.add(w) 
            mask_index.add(w)
            queue.append(w)
            visit_order[w] = next_visit_order
            
            if w == end_node:
                is_reach = True
                break
        
    if is_reach:
        break
            

logger.info(f'experiment: {EXP_NAME} time: {time.time() - t_start}')
logger.info(f'is_reach: {is_reach}') 
logger.info(f'masked:           {len(mask_index)}')
logger.info(f'last iteration:   {it}')

c_arr = v_arr.copy()
c_arr[:] = MYNULL
for pos_i,pos_j,pos_k in mask_index:
    c_arr[pos_i,pos_j,pos_k] = 500

updateVolumeFromArray(outa_node, c_arr)

c_arr = v_arr.copy()
c_arr[:] = MYNULL
for v,o in visit_order.items():
    pos_i,pos_j,pos_k = v
    c_arr[pos_i,pos_j,pos_k] = o*100

updateVolumeFromArray(outb_node, c_arr)


experiment: default EXP NAME time: 5.685369968414307
is_reach: True
masked:           77140
last iteration:   70962


In [11]:


## check z translation function 

# for v in range(1,275*512*512,7):
#     # i,j,k = z2coord(v)
#     # print(idx_arr_t[i,j,k])
#     if v != coord2z(*z2coord(v)):
#         print('broke')
#         break
# else:
    # print('work')

In [12]:


kernel_mask = np.zeros((5,5,5))
kernel_mask[1:4,1:4,1:4] = 1
kernel_mask[2,2,2] = 0
kernel_idxs = np.vstack(np.where(kernel_mask == 1)) - np.array([[2],[2],[2]])

# bfs find path
# seed = (162, 198, 313)
# end_node = (152, 175, 345)
queue = deque()

dist_grid = np.zeros_like(v_arr, np.uint16)
dist_grid[:] = 1e9

pred_grid = np.zeros_like(v_arr, np.int32)
pred_grid[:] = -1
queue.append(seed)
visited = set()

it = 0

visited.add(seed)
is_reach = False
while queue and it < 2e6:
    it += 1
    u = queue.popleft()
    pos = np.array(u)
    # print('pop', u)
    for pos_i in range(kernel_idxs.shape[1]):
        v = tuple((kernel_idxs[:,pos_i] + pos).tolist())
        if v not in visited and v in mask_index:
            
            visited.add(v)
            # dist_grid[v] = dist_grid[v] + 1
            vi, vj, vk = v
            pred_grid[vi,vj,vk] = coord2z(*u)
            queue.append(v)
            
            if (v == end_node):
                is_reach = True

                break
        # else:
        #     print('in visited')
            
    if is_reach:
        break

print(it)
print(is_reach)
pos_i,pos_j,pos_k = end_node
print(pred_grid[pos_i,pos_j,pos_k])
np.save('./data/pred_grid.npy', pred_grid)
    

66266
True
31789841


In [9]:
# i,j,k = end_node
pred_grid = np.load('./data/pred_grid.npy')
crawl = end_node
pos_i,pos_j,pos_k = crawl
path = []
it = 0
while  pred_grid[pos_i,pos_j,pos_k] != -1 and it < 512+275+512:
    it += 1
    parent = z2coord(pred_grid[pos_i,pos_j,pos_k])
    path.append(parent)
    crawl = parent
    pos_i,pos_j,pos_k = crawl
# print(path)

c_arr = v_arr.copy()
c_arr[:] = MYNULL
for order,(pos_i,pos_j,pos_k) in enumerate(path):
    # i,j,k = u
    c_arr[pos_i,pos_j,pos_k] = (order + 1) * 100
updateVolumeFromArray(outc_node, c_arr)
    
    

In [ ]:
np.sum(c_arr == 500)
it

In [14]:
seed

(160, 204, 306)

# Shortest Path

i >= 85 and j >= 115 and k >= 240 and i < 186 and j < 256 and k < 401

In [14]:
weight_arr = v_arr.copy()
boundi, boundj, boundk = weight_arr.shape

kernel_mask = np.zeros((5,5,5))
kernel_mask[1:4,1:4,1:4] = 1
kernel_mask[2,2,2] = 0
kernel_idxs = np.vstack(np.where(kernel_mask == 1)) - np.array([[2],[2],[2]])

# for i in range(kernel_idxs.shape[1]):
#     oi, oj, ok = kernel_idxs[:,i]
#     crystal_mask = np.zeros_like(cost_arr, dtype=np.bool)
#     crystal_mask[1:boundi-1, 1:boundj-1, 1:boundk-1] = v_arr[1+oi:boundi-1+oi, 1+oi:boundj-1+oi, 1+oi:boundk-1+oi] - v_arr[1:boundi-1, 1:boundj-1, 1:boundk-1] > 800 # check if near crystal
#     cost_arr[crystal_mask] = -1200
#     # print(np.sum(crystal_mask[85:186,115:256,240:401]))
        
    
# transform raw to cost
COST_OFFSET = -200

weight_arr[weight_arr > COST_OFFSET] = 2_000_000_000
weight_arr[weight_arr <= COST_OFFSET] += 4000
# cost_arr[:] = 100

In [38]:
updateVolumeFromArray(outc_node, cost_arr)

In [15]:
def kji_list2ras(kji_tuples):
    return kji2ras(np.array(kji_tuples).T)

def kji2ras(kji_3xN):
    return (ijk2ras_mat @ np.pad(kji_3xN[::-1,:], ((0,1), (0,0)), constant_values=1))[:3,:]

# x2 = kji_list2ras(full_path[:2])
# np.linalg.norm(x2[:,0] - x2[:,1]) * weight_arr[0,0,0]

In [104]:
u = (0,0,0)
u_pos = np.array(u)
for pos_i in range(kernel_idxs.shape[1]):
    v = tuple((kernel_idxs[:,pos_i] + u_pos).tolist())
    vi,vj,vk = v
    
    
    ras_vecx2 = kji_list2ras([u,v])
    dist_uv = np.linalg.norm(ras_vecx2[:,0] - ras_vecx2[:,1])
    print(dist_uv)

1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.1048543456039805
0.78125
1.1048543456039805
0.78125
0.78125
1.1048543456039805
0.78125
1.1048543456039805
1.6682934768798923
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.25
1.4740595518838444
1.6682934768798923
1.4740595518838444
1.6682934768798923


In [111]:
kernel_idxs

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [-1, -1, -1,  0,  0,  0,  1,  1,  1, -1, -1, -1,  0,  0,  1,  1,
         1, -1, -1, -1,  0,  0,  0,  1,  1,  1],
       [-1,  0,  1, -1,  0,  1, -1,  0,  1, -1,  0,  1, -1,  1, -1,  0,
         1, -1,  0,  1, -1,  0,  1, -1,  0,  1]])

In [118]:
u = (0,0,0)
u_pos = np.array(u)
for pos_i in range(kernel_idxs.shape[1]):
    v = tuple((kernel_idxs[:,pos_i] + u_pos).tolist())
    vi,vj,vk = v
    
    
    _uvk, _uvj, _uvi = kernel_idxs[:,pos_i] + 1
    dist_uv = kernel_ras_distance[_uvk, _uvj, _uvi]
    # print(_uvk,_uvj,_uvi)
    print(dist_uv)

1.6682935
1.4740596
1.6682935
1.4740596
1.25
1.4740596
1.6682935
1.4740596
1.6682935
1.1048543
0.78125
1.1048543
0.78125
0.78125
1.1048543
0.78125
1.1048543
1.6682935
1.4740596
1.6682935
1.4740596
1.25
1.4740596
1.6682935
1.4740596
1.6682935


In [16]:
ijk_spacing = volume_node.GetSpacing()
kernel_ras_distance = np.zeros((3,3,3), dtype=np.float32)
for pos_k in range(-1,2):
    for pos_j in range(-1,2):
        for pos_i in range(-1,2):
            kernel_ras_distance[pos_k+1,pos_j+1,pos_i+1] = ((pos_i*ijk_spacing[0])**2 + (pos_j*ijk_spacing[1])**2 + (pos_k*ijk_spacing[2])**2)**0.5
            # print(((i*ijk_spacing[0])**2 + (j*ijk_spacing[1])**2 + (k*ijk_spacing[2])**2)**0.5)
# dijkstra main function

def dijkstra(seed, end_node, weight_grid):
    kernel_mask = np.zeros((5,5,5))
    kernel_mask[1:4,1:4,1:4] = 1
    kernel_mask[2,2,2] = 0
    kernel_idxs = np.vstack(np.where(kernel_mask == 1)) - np.array([[2],[2],[2]])

    dist_grid = np.zeros_like(v_arr, np.float32)
    dist_grid[:] = 2e9
    dist_grid[seed] = 0

    pred_grid = np.full(v_arr.shape, -1, dtype=np.int32)

    pq = PriorityQueue()
    pq.put((0, seed))
    it = 0

    is_reach = False
    debug_dict = dict()

    while (not pq.empty()) and it < 1e6 and (not is_reach):
        it += 1
        
        u_dist, u = pq.get()
        ui,uj,uk = u
        u_pos = np.array(u)
        
        if u == end_node:
            is_reach = True
            break
        
        if u_dist > dist_grid[ui,uj,uk]:
            continue
        
        for i in range(kernel_idxs.shape[1]):
            v = tuple((kernel_idxs[:,i] + u_pos).tolist())
            vi,vj,vk = v
            
            # ras_vecx2 = kji_list2ras([u,v])
            # dist_uv = np.linalg.norm(ras_vecx2[:,0] - ras_vecx2[:,1])
            
            _uvk, _uvj, _uvi = kernel_idxs[:,i] + 1
            dist_uv = kernel_ras_distance[_uvk, _uvj, _uvi]
            
            # dist_uv = 1
            curr_cost = dist_uv * weight_grid[vi,vj,vk]
            dist = u_dist + curr_cost
            
            if dist < dist_grid[vi,vj,vk]:
                dist_grid[vi,vj,vk] = dist
                pred_grid[vi,vj,vk] = coord2z(*u)
                pq.put((dist, v))
            
            # if v == end_node:
            #     is_reach = True
    
    print(it)       
                
    if not is_reach:
        return []
    
    # get path inclusive [start --> end]
    crawl = end_node
    i,j,k = crawl
    path = [crawl]
    it = 0
    while  pred_grid[i,j,k] != -1 and it < 512+275+512:
        it += 1
        parent = z2coord(pred_grid[i,j,k])
        path.append(parent)
        crawl = parent
        i,j,k = crawl
    # path.append(crawl)
    # print(path)
    path.reverse()
    
    # return
    return path, debug_dict

t_start = time.time()
node_name = "LAD"
fid_node = getNodes(node_name)[node_name]
num_fid = fid_node.GetNumberOfFiducials()

seed = get_fiducial_as_kji(fid_node, 1)
end_node = get_fiducial_as_kji(fid_node, 2)
print(seed)
print(end_node)

path2, ddl2 = dijkstra(seed, end_node, weight_arr)
print(path2)

print(time.time() - t_start)

(162, 196, 317)
(160, 192, 326)
1608
[(162, 196, 317), (162, 196, 318), (161, 197, 319), (161, 197, 320), (161, 197, 321), (161, 197, 322), (161, 196, 323), (161, 195, 324), (160, 194, 325), (160, 193, 326), (160, 192, 326)]
0.42627573013305664


In [147]:
print(ddl1)
print(ddl2)


[1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.1048543, 0.78125, 1.1048543, 0.78125, 0.78125, 1.1048543, 0.78125, 1.1048543, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.1048543, 0.78125, 1.1048543, 0.78125, 0.78125, 1.1048543, 0.78125, 1.1048543, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.1048543, 0.78125, 1.1048543, 0.78125, 0.78125, 1.1048543, 0.78125, 1.1048543, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.6682935, 1.4740596, 1.6682935, 1.4740596, 1.25, 1.4740596, 1.6682935, 1.4740596, 1.6682935, 1.1048543, 0.78125, 1.1048543, 0.78125, 0.78125, 1.1048543, 0.78125, 1.1048543, 1.6682935, 1.4740596,

In [140]:
ddl2

[1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.25, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.1048543456039805, 0.78125, 1.1048543456039805, 0.78125, 0.78125, 1.1048543456039805, 0.78125, 1.1048543456039805, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.25, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.25, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.1048543456039805, 0.78125, 1.1048543456039805, 0.78125, 0.78125, 1.1048543456039805, 0.78125, 1.1048543456039805, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.25, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.6682934768798923, 1.4740595518838444, 1.6682934768798923, 1.4740595518838444, 1.25, 1.474059551883844

In [17]:
def calculate_backbone():
    t_start = time.time()
    EXP_NAME = "TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH"

    node_name = "LAD"
    fid_node = getNodes(node_name)[node_name]
    num_fid = fid_node.GetNumberOfFiducials()

    control_points_label2kji_map = {fid_node.GetNthFiducialLabel(i):np.array(get_fiducial_as_kji(fid_node, i)) for i in range(num_fid)}
    control_points_label2vis_map = {fid_node.GetNthFiducialLabel(i):fid_node.GetNthFiducialVisibility(i) for i in range(num_fid)}

    # filter non visible node
    for k,v in control_points_label2vis_map.items():
        if not v:
            control_points_label2kji_map.pop(k, None)

    # NN TSP sort
    first_node_label = 'LAD-1'
    last_node_label = 'LAD-9'

    # first_node_label = 'RCA-4'
    # last_node_label = 'RCA-12'
    
    unselected_pool = set(control_points_label2kji_map.keys())
    control_points_order = [first_node_label]
    unselected_pool.remove(first_node_label) 
    while len(unselected_pool):
        min_dist = 999999
        min_label = None
        current_node = control_points_label2kji_map[control_points_order[-1]]
        for label in unselected_pool:
            neighbor_node = control_points_label2kji_map[label]
            diff = current_node - neighbor_node
            dist = np.dot(diff, diff)
            if dist < min_dist:
                min_dist = dist
                min_label = label
        unselected_pool.remove(min_label)
        control_points_order.append(min_label)
        
        if min_label == last_node_label:
            break

    print(control_points_order)    
    control_points_kji = [tuple(control_points_label2kji_map[e].tolist()) for e in control_points_order]

    # find shortest path

    sub_paths = []
    for i in range(len(control_points_kji) - 1):
        # seed = get_fiducial_as_kji(fid_node, i)
        # end_node = get_fiducial_as_kji(fid_node, i+1)
        seed = control_points_kji[i]
        end_node = control_points_kji[i+1]
        sub_path, _ = dijkstra(seed, end_node, weight_arr)
        if sub_path is not None:
            sub_paths.append(sub_path)

    for sub_path in sub_paths:
        print(sub_path)
    
    full_path = []
    c_arr = v_arr.copy()
    c_arr[:] = MYNULL
    
    for e in sub_paths:
        for order,(i,j,k) in enumerate(e):
            # i,j,k = u
            c_arr[i,j,k] = (order + 1) * 100
        full_path += e[0:-1]
    full_path.append(sub_paths[-1][-1])

    # for order,(i,j,k) in enumerate():
    #         # i,j,k = u
    #         c_arr[i,j,k] = 500

    updateVolumeFromArray(outc_node, c_arr)

    ###### logging result
    logger.info(f'experiment: {EXP_NAME} time: {time.time() - t_start}')
    # logger.info(f'is_reach: {is_reach}') 
    logger.info(f'len path:           {len(full_path)}')
    # logger.info(f'last iteration:   {it}')
    # logger.info(f'{mycount}')
    
    return control_points_order, control_points_kji, full_path, sub_paths
# control_points_order, control_points_kji, full_path = calculate_backbone()

In [13]:
t_start = time.time()
EXP_NAME = "TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH"

node_name = "RCA"
fid_node = getNodes(node_name)[node_name]
num_fid = fid_node.GetNumberOfFiducials()

control_points_label2kji_map = {fid_node.GetNthFiducialLabel(i):np.array(get_fiducial_as_kji(fid_node, i)) for i in range(num_fid)}
control_points_label2vis_map = {fid_node.GetNthFiducialLabel(i):fid_node.GetNthFiducialVisibility(i) for i in range(num_fid)}

# filter non visible node
for pos_k,v in control_points_label2vis_map.items():
    if not v:
        control_points_label2kji_map.pop(pos_k, None)

# NN TSP sort
# first_node_label = 'LAD-1'
# last_node_label = 'LAD-9'

first_node_label = 'RCA-4'
last_node_label = 'RCA-12'

# first_node_label = 'LCX-1'
# last_node_label = 'LCX-8'


unselected_pool = set(control_points_label2kji_map.keys())
control_points_order = [first_node_label]
unselected_pool.remove(first_node_label) 
while len(unselected_pool):
    min_dist = 999999
    min_label = None
    current_node = control_points_label2kji_map[control_points_order[-1]]
    for label in unselected_pool:
        neighbor_node = control_points_label2kji_map[label]
        diff = current_node - neighbor_node
        dist = np.dot(diff, diff)
        if dist < min_dist:
            min_dist = dist
            min_label = label
    unselected_pool.remove(min_label)
    control_points_order.append(min_label)
    
    if min_label == last_node_label:
        break

print(control_points_order)    
control_points_kji = [tuple(control_points_label2kji_map[e].tolist()) for e in control_points_order]

# find shortest path

sub_paths = []
for pos_i in range(len(control_points_kji) - 1):
    # seed = get_fiducial_as_kji(fid_node, i)
    # end_node = get_fiducial_as_kji(fid_node, i+1)
    seed = control_points_kji[pos_i]
    end_node = control_points_kji[pos_i+1]
    sub_path = dijkstra(seed, end_node, cost_arr)
    if sub_path is not None:
        sub_paths.append(sub_path)

full_path = []
c_arr = v_arr.copy()
c_arr[:] = MYNULL
 
for e in sub_paths:
    for order,(pos_i,pos_j,pos_k) in enumerate(e):
        # i,j,k = u
        c_arr[pos_i,pos_j,pos_k] = (order + 1) * 100
    full_path += e[0:-1]
full_path.append(sub_paths[-1][-1])

# for order,(i,j,k) in enumerate():
#         # i,j,k = u
#         c_arr[i,j,k] = 500

updateVolumeFromArray(outc_node, c_arr)

###### logging result
logger.info(f'experiment: {EXP_NAME} time: {time.time() - t_start}')
# logger.info(f'is_reach: {is_reach}') 
logger.info(f'len path:           {len(full_path)}')
# logger.info(f'last iteration:   {it}')
# logger.info(f'{mycount}')


['RCA-4', 'RCA-5', 'RCA-6', 'RCA-7', 'RCA-8', 'RCA-9', 'RCA-1', 'RCA-10', 'RCA-2', 'RCA-3', 'RCA-13', 'RCA-14', 'RCA-11', 'RCA-16', 'RCA-15', 'RCA-12']
9161
5
103
1039
39
9
37
88
99
664
20
6
5
8
6


experiment: TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH time: 30.37398886680603
len path:           47


NameError: name 'mycount' is not defined

In [70]:
print(len(full_path))
print(len(set(full_path)))

10401
45


In [51]:
# LAD
seed = (162, 198, 313)
end_node = (152, 175, 345)

# # LCX
# seed = (160, 204, 306)
# end_node = (151, 212, 303)

# # RCA
# seed = (93, 156, 282)
# end_node = (122, 138, 274)

kernel_mask = np.zeros((5,5,5))
kernel_mask[1:4,1:4,1:4] = 1
kernel_mask[2,2,2] = 0
kernel_idxs = np.vstack(np.where(kernel_mask == 1)) - np.array([[2],[2],[2]])

# dijkstra
EXP_NAME = "DIJKSTRA SHORTEST PATH"
t_start = time.time()

dist_grid = np.zeros_like(v_arr, np.uint32)
dist_grid[:] = 2e9

pred_grid = np.full(v_arr.shape, -1, dtype=np.int32)

pq = PriorityQueue()
pq.put((0, seed))
it = 0

is_reach = False

mycount = 0


while (not pq.empty()) and it < 1e6 and (not is_reach):
    it += 1
    
    u_dist, u = pq.get()
    ui,uj,uk = u
    u_pos = np.array(u)
    
    if u_dist > dist_grid[ui,uj,uk]:
        mycount += 1
        continue
    
    for pos_i in range(kernel_idxs.shape[1]):
        v = tuple((kernel_idxs[:,pos_i] + u_pos).tolist())
        vi,vj,vk = v
        
        dist = u_dist + cost_arr[vi,vj,vk]
        
        if dist < dist_grid[vi,vj,vk]:
            dist_grid[vi,vj,vk] = dist
            pred_grid[vi,vj,vk] = coord2z(*u)
            pq.put((dist, v))
        
        if v == end_node:
            is_reach = True
            break

logger.info(f'experiment: {EXP_NAME} time: {time.time() - t_start}')
logger.info(f'is_reach: {is_reach}') 
logger.info(f'masked:           {np.sum(pred_grid != -1)}')
logger.info(f'last iteration:   {it}')
logger.info(f'{mycount}')

# print path
crawl = end_node
pos_i,pos_j,pos_k = crawl
path = []
it = 0
while  pred_grid[pos_i,pos_j,pos_k] != -1 and it < 512+275+512:
    it += 1
    parent = z2coord(pred_grid[pos_i,pos_j,pos_k])
    path.append(parent)
    crawl = parent
    pos_i,pos_j,pos_k = crawl
print(path)

c_arr = v_arr.copy()
c_arr[:] = MYNULL
for order,(pos_i,pos_j,pos_k) in enumerate(path):
    # i,j,k = u
    c_arr[pos_i,pos_j,pos_k] = (order + 1) * 100
updateVolumeFromArray(outc_node, c_arr)

       

    
    
    
    


experiment: DIJKSTRA SHORTEST PATH time: 7.601354122161865
is_reach: True
masked:           128822
last iteration:   93178
0


[(152, 176, 344), (152, 177, 343), (153, 178, 342), (153, 179, 341), (154, 180, 340), (154, 181, 339), (155, 182, 338), (155, 183, 337), (156, 184, 336), (156, 185, 335), (157, 186, 334), (157, 187, 333), (158, 188, 332), (158, 189, 331), (159, 190, 330), (159, 191, 329), (159, 191, 328), (160, 192, 327), (160, 192, 326), (161, 193, 325), (161, 194, 324), (161, 195, 323), (162, 195, 322), (163, 194, 321), (164, 193, 320), (164, 194, 319), (164, 195, 318), (165, 196, 317), (165, 196, 316), (164, 197, 315), (163, 198, 314), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 31

# Create Curve

In [14]:
def calculate_backbone2(first_node_label, last_node_label, node_name):
    t_start = time.time()
    EXP_NAME = "TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH"

    fid_node = getNodes(node_name)[node_name]
    num_fid = fid_node.GetNumberOfFiducials()

    control_points_label2kji_map = {fid_node.GetNthFiducialLabel(i):np.array(get_fiducial_as_kji(fid_node, i)) for i in range(num_fid)}
    control_points_label2vis_map = {fid_node.GetNthFiducialLabel(i):fid_node.GetNthFiducialVisibility(i) for i in range(num_fid)}

    # filter non visible node
    for k,v in control_points_label2vis_map.items():
        if not v:
            control_points_label2kji_map.pop(k, None)

    # NN TSP sort
    # first_node_label = 'LAD-1'
    # last_node_label = 'LAD-9'

    # first_node_label = 'RCA-4'
    # last_node_label = 'RCA-12'
    
    unselected_pool = set(control_points_label2kji_map.keys())
    control_points_order = [first_node_label]
    unselected_pool.remove(first_node_label) 
    while len(unselected_pool):
        min_dist = 999999
        min_label = None
        current_node = control_points_label2kji_map[control_points_order[-1]]
        for label in unselected_pool:
            neighbor_node = control_points_label2kji_map[label]
            diff = current_node - neighbor_node
            dist = np.dot(diff, diff)
            if dist < min_dist:
                min_dist = dist
                min_label = label
        unselected_pool.remove(min_label)
        control_points_order.append(min_label)
        
        if min_label == last_node_label:
            break

    print(control_points_order)    
    control_points_kji = [tuple(control_points_label2kji_map[e].tolist()) for e in control_points_order]

    # find shortest path

    sub_paths = []
    for i in range(len(control_points_kji) - 1):
        # seed = get_fiducial_as_kji(fid_node, i)
        # end_node = get_fiducial_as_kji(fid_node, i+1)
        seed = control_points_kji[i]
        end_node = control_points_kji[i+1]
        sub_path, _ = dijkstra(seed, end_node, weight_arr)
        if sub_path is not None:
            sub_paths.append(sub_path)

    for sub_path in sub_paths:
        print(sub_path)
    
    full_path = []
    
    for e in sub_paths:
        full_path += e[0:-1]
    full_path.append(sub_paths[-1][-1])

    # for order,(i,j,k) in enumerate():
    #         # i,j,k = u
    #         c_arr[i,j,k] = 500

    ###### logging result
    logger.info(f'experiment: {EXP_NAME} time: {time.time() - t_start}')
    # logger.info(f'is_reach: {is_reach}') 
    logger.info(f'len path:           {len(full_path)}')
    # logger.info(f'last iteration:   {it}')
    # logger.info(f'{mycount}')
    
    return control_points_order, control_points_kji, full_path, sub_paths
# control_points_order, control_points_kji, full_path = calculate_backbone()

In [16]:
# def create_curve_all_point(points_kji, curve_name):
#     x = np.array(points_kji, np.float)
#     x = x.T
#     x = x[::-1,:]
#     x = np.concatenate( [x, np.ones((1, x.shape[1]))] ,axis=0)
#     x = ijk2ras_mat @ x
#     x = x[:3,:]

#     curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
#     curve_node.SetName(curve_name)
    
#     for pos_j in range(x.shape[1]):
#         curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,pos_j]))
        
#     return curve_node

def create_curve(points_kji, curve_name, spacing=0.4):
    x = np.array(points_kji, np.float)
    x = x.T
    x = x[::-1,:]
    x = np.concatenate( [x, np.ones((1, x.shape[1]))] ,axis=0)
    x = ijk2ras_mat @ x
    x = x[:3,:]
    
    curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
    curve_node.SetName(curve_name)

    # TODO optimize to vectorize accumulation
    CURVE_CTRL_SPACING = spacing # may be mm
    count = 0
    last_add_dist = 0
    curr_dist = 0
    curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,0]))

    for pos_j in range(1, x.shape[1]):
        prev_ras = x[:,pos_j-1]
        curr_ras = x[:,pos_j]
        curr_dist += np.linalg.norm(curr_ras - prev_ras)
        if curr_dist - last_add_dist >= CURVE_CTRL_SPACING:
            count += 1
            # print(curr_dist - last_add_dist)
            curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,pos_j]))
            last_add_dist = curr_dist
            
    return curve_node

# curve_name = "LAD_CONTROLPOINTS_CURVE"
# curve = create_curve(control_points_kji, curve_name, 0)
# print(f'{curve_name} - length: {curve.GetCurveLengthWorld():.2f}')
    

In [17]:
# first_node_label = 'LAD-1'
# last_node_label = 'LAD-9'

# first_node_label = 'RCA-4'
# last_node_label = 'RCA-12'

# first_node_label = 'LCX-1'
# last_node_label = 'LCX-8'


curve_data = dict()
        
for first_node_label, last_node_label, node_name in zip(('LAD-1', 'RCA-4', 'LCX-1'), 
                                             ('LAD-9', 'RCA-12', 'LCX-8'), 
                                             ('LAD', 'RCA', 'LCX')):
    
    control_points_order, control_points_kji, full_path, sub_paths = calculate_backbone2(first_node_label, last_node_label, node_name)
    
    curve_name = f"{node_name}_CONTROLPOINTS_CURVE"
    curve = create_curve(control_points_kji, curve_name, 0)
    
    curve_data[curve_name] = (control_points_order, control_points_kji, full_path, sub_paths, curve)
    
    # control_points_order, control_points_kji, full_path, sub_paths = calculate_backbone2(first_node_label, last_node_label, node_name)
    
    curve_name = f"{node_name}_FINE_CURVE"
    curve = create_curve(full_path, curve_name, 0.4)
    
    curve_data[curve_name] = (control_points_order, control_points_kji, full_path, sub_paths, curve)
    # print(f'{curve_name} - length: {curve.GetCurveLengthWorld():.2f}')

# c_arr = v_arr.copy()
# c_arr[:] = MYNULL
# for e in sub_paths:
#     for order,(i,j,k) in enumerate(e):
#     # i,j,k = u
#     c_arr[i,j,k] = (order + 1) * 100

# updateVolumeFromArray(outc_node, c_arr)

['LAD-1', 'LAD-2', 'LAD-3', 'LAD-4', 'LAD-5', 'LAD-6', 'LAD-7', 'LAD-8', 'LAD-9']


55
1608
45
37
28
59
29
28
[(162, 198, 313), (162, 198, 314), (162, 198, 315), (162, 197, 316), (162, 196, 317)]
[(162, 196, 317), (162, 196, 318), (161, 197, 319), (161, 197, 320), (161, 197, 321), (161, 197, 322), (161, 196, 323), (161, 195, 324), (160, 194, 325), (160, 193, 326), (160, 192, 326)]
[(160, 192, 326), (160, 192, 327), (160, 191, 328), (159, 190, 329)]
[(159, 190, 329), (159, 190, 330), (159, 190, 331), (158, 189, 332), (158, 188, 332)]
[(158, 188, 332), (157, 187, 333), (157, 186, 334)]
[(157, 186, 334), (156, 185, 335), (156, 184, 335), (156, 183, 336), (155, 182, 337)]
[(155, 182, 337), (155, 182, 338), (154, 181, 339), (154, 180, 340), (153, 179, 341)]
[(153, 179, 341), (153, 178, 342), (152, 177, 343), (152, 177, 344), (152, 176, 345), (152, 175, 345)]


experiment: TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH time: 7.150854825973511
len path:           37


['RCA-4', 'RCA-5', 'RCA-6', 'RCA-7', 'RCA-8', 'RCA-9', 'RCA-1', 'RCA-10', 'RCA-2', 'RCA-3', 'RCA-13', 'RCA-14', 'RCA-11', 'RCA-16', 'RCA-15', 'RCA-12']
6653
10
193
1967
74
13
57
226
164
1831
96
11
12
21
10
[(93, 156, 282), (93, 155, 282), (93, 154, 281), (93, 154, 280), (93, 153, 279), (93, 152, 278), (93, 151, 277), (93, 150, 276), (93, 149, 276), (93, 148, 276), (93, 147, 275), (93, 146, 274), (93, 145, 274), (93, 144, 274), (93, 143, 274), (93, 142, 274)]
[(93, 142, 274), (94, 142, 274)]
[(94, 142, 274), (94, 141, 274), (95, 140, 273), (96, 139, 274)]
[(96, 139, 274), (96, 138, 274), (96, 137, 274), (96, 136, 274), (97, 135, 274), (98, 134, 275), (99, 133, 274), (100, 133, 273)]
[(100, 133, 273), (101, 132, 274), (102, 132, 274)]
[(102, 132, 274), (103, 131, 274)]
[(103, 131, 274), (104, 131, 274), (105, 132, 274)]
[(105, 132, 274), (106, 133, 274), (107, 133, 274), (108, 134, 273)]
[(108, 134, 273), (109, 134, 273), (110, 134, 273), (111, 134, 273)]
[(111, 134, 273), (112, 133, 273

experiment: TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH time: 21.308057069778442
len path:           49


['LCX-1', 'LCX-2', 'LCX-3', 'LCX-4', 'LCX-5', 'LCX-6', 'LCX-8']
59
49
24
943424
8
14
[(160, 204, 307), (159, 204, 307), (158, 204, 306)]
[(158, 204, 306), (157, 205, 305), (156, 205, 304)]
[(156, 205, 304), (155, 206, 303), (154, 206, 303)]
[(154, 206, 303), (155, 205, 304), (156, 206, 305), (157, 206, 305), (158, 206, 305), (159, 205, 306), (159, 204, 306), (160, 203, 306), (161, 202, 307), (162, 201, 308), (162, 201, 309), (162, 200, 310), (163, 199, 311), (163, 198, 312), (164, 197, 313), (164, 197, 314), (164, 197, 315), (164, 196, 316), (164, 195, 317), (164, 194, 318), (163, 193, 319), (163, 193, 320), (163, 193, 321), (163, 193, 322), (162, 194, 323), (161, 195, 324), (160, 196, 325), (160, 196, 326), (160, 197, 327), (160, 197, 328), (160, 197, 329), (160, 198, 330), (160, 198, 331), (160, 198, 332), (160, 198, 333), (159, 199, 334), (159, 200, 333), (159, 201, 332), (159, 202, 331), (158, 203, 330), (158, 203, 329), (158, 203, 328), (157, 204, 327), (157, 205, 326), (157, 206,

experiment: TSP + MULTI SUBPATH + DIJKSTRA SHORTEST PATH time: 614.7667970657349
len path:           80


In [19]:
# addendum
for artery_name in ('LAD', 'RCA', 'LCX'):
    curve_name = f'{artery_name}_FINE_CURVE'
    control_points_order, control_points_kji, full_path, sub_paths, _ = curve_data[curve_name]
    curve_data[curve_name] = control_points_order, control_points_kji, full_path, sub_paths, create_curve(full_path, curve_name, 0.4)

In [24]:
for curve_name in curve_data.keys():
    curve_node = curve_data[curve_name][-1]
    print(f'{curve_name: <30} - length: {curve_node.GetCurveLengthWorld():.2f}')

LAD_CONTROLPOINTS_CURVE        - length: 34.26
LAD_FINE_CURVE                 - length: 41.85
RCA_CONTROLPOINTS_CURVE        - length: 57.58
RCA_FINE_CURVE                 - length: 61.39
LCX_CONTROLPOINTS_CURVE        - length: 14.50
LCX_FINE_CURVE                 - length: 98.78


In [ ]:
print(f'only control point curve - length: {ctrlpts_curve_node.GetCurveLengthWorld():.3f} mm.')
print(f'fine curve - length: {allpts_curve_node.GetCurveLengthWorld():.3f} mm.')

In [21]:
len(curve_data['RCA_FINE_CURVE'][2])

49

In [36]:
np.save('data/pda/full_path.npy', np.array(full_path))

In [39]:
# sanity check
count = 0
for pos_i,sub_path in enumerate(sub_paths):
    unique_node = len(set(sub_path))
    print(f'path {pos_i}: {len(sub_path)} -uniq-> {unique_node}')
    count += unique_node
print(f'unique node: {count}')
    

path 0: 5 -uniq-> 5
path 1: 11 -uniq-> 11
path 2: 4 -uniq-> 4
path 3: 5 -uniq-> 5
path 4: 3 -uniq-> 3
path 5: 5 -uniq-> 5
path 6: 5 -uniq-> 5
path 7: 6 -uniq-> 6
unique node: 44


In [16]:
print(len(full_path))
print(len(set(full_path)))

34
34


In [45]:
print(len(sub_paths))
for pos_i,sub_path in enumerate(sub_paths):
    curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
    curve_node.SetName(f'debug_curve_{pos_i}')
    x = np.array(sub_path, np.float)
    x = x.T
    x = x[::-1,:]
    x = np.concatenate( [x, np.ones((1, x.shape[1]))] ,axis=0)
    x = ijk2ras_mat @ x
    x = x[:3,:]

    for pos_j in range(0, x.shape[1], 8):
        curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,pos_j]))
    


8


In [41]:
ctrlpts_curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
ctrlpts_curve_node.SetName('control_points_curve')

allpts_curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
allpts_curve_node.SetName('all_points_curve')

In [21]:
ras2ijk_mat @ np.concatenate([nodes_pos[pos_i], 
                np.ones((nodes_pos[pos_i].shape[0],1))], axis=1).T

[(162, 198, 313), (162, 196, 317), (160, 192, 326), (159, 190, 329), (158, 188, 332), (157, 186, 334), (155, 182, 337), (153, 179, 341), (152, 175, 345)]

In [21]:
# create curve
for e in control_points_kji:
    # ctrlpts_curve_node.GetNthControlPointPosition(i, position)
    ctrlpts_curve_node.AddControlPoint(vtk.vtkVector3d(*e))
    
# for e in full_path:
#     allpts_curve_node.AddControlPoint(vtk.vtkVector3d(*e))

In [44]:
x = np.array(control_points_kji, np.float)
x = x.T
x = x[::-1,:]
x = np.concatenate( [x, np.ones((1, x.shape[1]))] ,axis=0)
x = ijk2ras_mat @ x
x = x[:3,:]

for pos_j in range(x.shape[1]):
    
    ctrlpts_curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,pos_j]))
# for i in range()

# x = np.array(ijk2kji(x))

In [42]:
x = np.array(full_path, np.float)
x = x.T
x = x[::-1,:]
x = np.concatenate( [x, np.ones((1, x.shape[1]))] ,axis=0)
x = ijk2ras_mat @ x
x = x[:3,:]


# TODO optimize to vectorize accumulation
CURVE_CTRL_SPACING = 0.4 # may be mm
count = 0
last_add_dist = 0
curr_dist = 0
allpts_curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,0]))

for pos_j in range(1, x.shape[1]):
    prev_ras = x[:,pos_j-1]
    curr_ras = x[:,pos_j]
    curr_dist += np.linalg.norm(curr_ras - prev_ras)
    if curr_dist - last_add_dist >= CURVE_CTRL_SPACING:
        count += 1
        # print(curr_dist - last_add_dist)
        allpts_curve_node.AddControlPoint(vtk.vtkVector3d(*x[:,pos_j]))
        last_add_dist = curr_dist
print(count)
print(curr_dist)


36
40.55508257044272


In [49]:
print(f'only control point curve - length: {ctrlpts_curve_node.GetCurveLengthWorld():.3f} mm.')
print(f'fine curve - length: {allpts_curve_node.GetCurveLengthWorld():.3f} mm.')

only control point curve - length: 34.265 mm.
fine curve - length: 41.846 mm.


# Flood Fill with caps

In [43]:
for ctrlpts in control_points_kji:
    print(ctrlpts)
    
    allpts_curve_node.GetCurveDirectionAtPointIndexWorld()
    allpts_curve_node.

(162, 198, 313)
(162, 196, 317)
(160, 192, 326)
(159, 190, 329)
(158, 188, 332)
(157, 186, 334)
(155, 182, 337)
(153, 179, 341)
(152, 175, 345)


In [40]:
allpts_curve_node.

NameError: name 'allpts_curve_node' is not defined

# Create Fid


In [12]:
#
x = getNode("myFMarkups")

In [57]:


# Create a fiducial node
fiducial_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')
fiducial_node.SetName('MyFiducials')



In [7]:
curve_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsCurveNode')
curve_node.SetName('MyCurve')

In [17]:
n = curve.GetNumberOfControlPoints()
position = [0,0,0]
for pos_i in range(n):
    curve.GetNthControlPointPosition(pos_i, position)
    curve_node.AddControlPoint(vtk.vtkVector3d(*position))
    print(position)

[54.73856209150327, -24.509803921568626, 0.0]
[11.710239651416117, 21.786492374727672, 0.0]
[-38.943355119825696, -17.973856209150327, 0.0]
[-2.4509803921568647, -56.10021786492375, 0.0]
[-53.649237472766885, -69.71677559912854, 0.0]


In [14]:
type(fiducial_node)

<class 'vtkSlicerMarkupsModuleMRMLPython.vtkMRMLMarkupsFiducialNode'>

In [15]:
type(x)

<class 'vtkSlicerMarkupsModuleMRMLPython.vtkMRMLMarkupsFiducialNode'>

In [16]:
fiducial_node.AddFiducialFromArray(np.array([40, 50, 60]))

18

In [58]:
nodes_pos

[array([[ -21.5655673 ,   45.4757882 , -386.5       ],
       [ -25.11234184,   46.630552  , -386.5       ],
       [ -31.71099214,   50.13608498, -389.        ],
       [ -34.31844552,   51.38546313, -390.25      ],
       [ -36.26009046,   53.11854243, -391.5       ],
       [ -37.69393932,   54.9108535 , -392.75      ],
       [ -40.44214962,   57.77855121, -395.25      ],
       [ -43.54882214,   60.52676152, -397.75      ],
       [ -46.47626356,   63.21522812, -399.        ]]), array([[ -16.61776477,   40.89474248, -389.        ],
       [ -15.94718328,   40.26251835, -391.5       ],
       [ -14.97580568,   39.73946888, -394.        ],
       [ -13.85498537,   38.99225534, -396.5       ],
       [ -13.63082131,   37.79671368, -399.        ],
       [ -14.00442808,   36.45172931, -400.25      ],
       [ -13.22317808,   35.30209255, -400.73523933],
       [ -13.9287411 ,   34.88922931, -400.72291994]]), array([[   8.7076287 ,   97.40301089, -460.25      ],
       [   9.78533142, 

In [64]:
# #Combine multiple fid to one fid
    
# LAD_fid_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')
# LAD_fid_node.SetName('LAD')
# multi_add_fiducial(LAD_fid_node, nodes_pos[0])

# LCX_fid_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')
# LCX_fid_node.SetName('LCX')
# multi_add_fiducial(LCX_fid_node, nodes_pos[1])

# RCA_fid_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLMarkupsFiducialNode')
# RCA_fid_node.SetName('RCA')
# multi_add_fiducial(RCA_fid_node, nodes_pos[2])

In [37]:
# # import slicer
# # from slicer.util import VTKObservationMixin

# class AnimationController(VTKObservationMixin):
#     def __init__(self, parent=None):
#         VTKObservationMixin.__init__(self)
#         self.parent = parent
#         self.slider = None
#         self.animation_logic = slicer.modules.animatedslices.logic()

#     def create_ui(self):
#         # Create slider
#         self.slider = slicer.qMRMLSliderWidget()
#         self.slider.minimum = 0
#         self.slider.maximum = 100
#         self.slider.value = 0
#         self.slider.setDecimals(0)
#         self.slider.singleStep = 1
#         self.slider.setToolTip("Animate slices")
#         self.parent.layout().addWidget(self.slider)

#         # Connect slider value change to animation logic
#         self.addObserver(self.slider, slicer.qMRMLSliderWidget.ValueChangedEvent, self.on_slider_value_changed)

#     def on_slider_value_changed(self, value):
#         # Update animation frame
#         frame = int(value)
#         self.animation_logic.setCurrentFrame(frame)

# # Instantiate the AnimationController
# controller = AnimationController(slicer.util.mainWindow())

# # Create the UI
# controller.create_ui()

# # Display the main window
# slicer.util.mainWindow().show()
pass

# Module Development

In [36]:
module_names = moduleNames()
print('\n'.join(module_names[:10]))
print('number of module name:',len(module_names))
print('\n'.join(module_names), file=open('module_names.txt', 'w'))

ACPCTransform
AddManyMarkupsFiducialTest
AddScalarVolumes
SubjectHierarchy
Annotations
AtlasTests
BRAINSDWICleanup
BRAINSFit
BRAINSFitRigidRegistrationCrashIssue4139
BRAINSLabelStats
number of module name: 185


In [38]:
mod_ExtractCenterline = getModuleWidget('ExtractCenterline')
mod_ExtractCenterline.__dict__


{'moduleName': 'ExtractCenterline', 'developerMode': False, 'parent': qSlicerScriptedLoadableModuleWidget(0x7fe3864f1c40, name="ExtractCenterlineModuleWidget") , 'layout': QVBoxLayout (QVBoxLayout at: 0x7fe385dae2c0), 'Observations': [[(MRMLCorePython.vtkMRMLScriptedModuleNode)0x1b0fe92e8, 36, <bound method ExtractCenterlineWidget.updateGUIFromParameterNode of <ExtractCenterline.ExtractCenterlineWidget object at 0x1b1495e48>>, 'none', 7, 0.0]], 'logic': <ExtractCenterline.ExtractCenterlineLogic object at 0x1b14956d8>, '_parameterNode': (MRMLCorePython.vtkMRMLScriptedModuleNode)0x1b0fe92e8, 'updatingGUIFromParameterNode': False, 'ui': <slicer.util. object at 0x1b14953c8>, 'nodeSelectors': [(qMRMLNodeComboBox(0x7fe3862e1950, name="inputSurfaceSelector") , 'InputSurface'), (qMRMLNodeComboBox(0x7fe3862f60c0, name="endPointsMarkupsSelector") , 'EndPoints'), (qMRMLNodeComboBox(0x7fe386296b20, name="outputNetworkModelSelector") , 'NetworkModel'), (qMRMLNodeComboBox(0x7fe314cb1260, name="outpu

In [42]:
mod_logic_ExtractCenterline = getModuleLogic('ExtractCenterline')
mod_logic_ExtractCenterline.__dict__

{'moduleName': 'ExtractCenterline', 'isSingletonParameterNode': True, 'enableScreenshots': False, 'screenshotScaleFactor': 1.0, 'blankingArrayName': 'Blanking', 'radiusArrayName': 'Radius', 'groupIdsArrayName': 'GroupIds', 'centerlineIdsArrayName': 'CenterlineIds', 'tractIdsArrayName': 'TractIds', 'topologyArrayName': 'Topology', 'marksArrayName': 'Marks', 'lengthArrayName': 'Length', 'curvatureArrayName': 'Curvature', 'torsionArrayName': 'Torsion', 'tortuosityArrayName': 'Tortuosity', 'frenetTangentArrayName': 'FrenetTangent', 'frenetNormalArrayName': 'FrenetNormal', 'frenetBinormalArrayName': 'FrenetBinormal'}

In [39]:
mod_CrossSectionAnalysis = getModuleWidget('CrossSectionAnalysis')
mod_CrossSectionAnalysis.__dict__

{'moduleName': 'CrossSectionAnalysis', 'developerMode': False, 'parent': qSlicerScriptedLoadableModuleWidget(0x7fe314cae5e0, name="CrossSectionAnalysisModuleWidget") , 'layout': QVBoxLayout (QVBoxLayout at: 0x7fe3864b9110), 'Observations': [[(MRMLCorePython.vtkMRMLScriptedModuleNode)0x1b73fc228, 36, <bound method CrossSectionAnalysisWidget.updateGUIFromParameterNode of <CrossSectionAnalysis.CrossSectionAnalysisWidget object at 0x1b7430d30>>, 'none', 7, 0.0], [(MRMLCorePython.vtkMRMLScene)0x1520af2e8, 8451, <bound method CrossSectionAnalysisWidget.onSceneStartClose of <CrossSectionAnalysis.CrossSectionAnalysisWidget object at 0x1b7430d30>>, 'none', 2247, 0.0], [(MRMLCorePython.vtkMRMLScene)0x1520af2e8, 8707, <bound method CrossSectionAnalysisWidget.onSceneEndClose of <CrossSectionAnalysis.CrossSectionAnalysisWidget object at 0x1b7430d30>>, 'none', 2248, 0.0]], 'logic': <CrossSectionAnalysis.CrossSectionAnalysisLogic object at 0x1b7430da0>, '_parameterNode': (MRMLCorePython.vtkMRMLScript

In [54]:
print(len(full_path))
print(len(set(full_path)))

10401
39


In [55]:
full_path

[(163, 198, 312), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 313), (163, 198, 312), (162, 198, 31